In [99]:
from pandas import read_sql
import plotly.graph_objs as go
import pandas as pd
import os
import json

In [100]:
# Start and end times
START = "2025-05-29 11:07:50"
END = "2025-05-29 11:08:05"

QUERY = f"""select * from heart_rate where time > '{START}' and time < '{END}' order by time asc"""

In [101]:
DATABASE_URL = os.getenv("DATABASE_URL").replace("sqlite://", "sqlite:///../")
df = read_sql(QUERY, DATABASE_URL)

In [102]:
df['time'] = pd.to_datetime(df['time'])

In [103]:
imu_records = []

position = 0
for idx, row in df.iterrows():
    base_time = row['time']
    imu_list = json.loads(row['imu_data'])
    for i, entry in enumerate(imu_list):
        timestamp = base_time + pd.to_timedelta(i, unit='ms')  # Adjust time resolution if needed
        imu_records.append({
            'timestamp': position,
            'acc_x': entry['acc_x_g'],
            'acc_y': entry['acc_y_g'],
            'acc_z': entry['acc_z_g'],
            'gry_x': entry['gyr_x_dps'],
            'gry_y': entry['gyr_y_dps'],
            'gry_z': entry['gyr_z_dps'],
        })
        position += 1

imu_df = pd.DataFrame(imu_records)

fig = go.Figure()

# Acceleration traces on y-axis 1 (default)
fig.add_trace(go.Scatter(x=imu_df['timestamp'], y=imu_df['acc_x'], name='Acc X', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=imu_df['timestamp'], y=imu_df['acc_y'], name='Acc Y', line=dict(color='green')))
fig.add_trace(go.Scatter(x=imu_df['timestamp'], y=imu_df['acc_z'], name='Acc Z', line=dict(color='red')))

# Gyroscope traces on y-axis 2 (right side)
fig.add_trace(go.Scatter(x=imu_df['timestamp'], y=imu_df['gry_x'], name='Gry X', yaxis='y2', line=dict(color='purple', dash='dot')))
fig.add_trace(go.Scatter(x=imu_df['timestamp'], y=imu_df['gry_y'], name='Gry Y', yaxis='y2', line=dict(color='orange', dash='dot')))
fig.add_trace(go.Scatter(x=imu_df['timestamp'], y=imu_df['gry_z'], name='Gry Z', yaxis='y2', line=dict(color='brown', dash='dot')))

fig.update_layout(
    title='IMU Data (Acceleration & Gyroscope)',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Acceleration (m/s²)', side='left'),
    yaxis2=dict(
        title='Gyroscope (rad/s)',
        overlaying='y',
        side='right'
    ),
    legend=dict(title='Sensor Axis'),
    template='plotly_white'
)

fig.show()